# Multilayer LSTM

## Load the dataset

In [1]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

tokenizer = info.features['text'].encoder

c:\Users\BACHLONG\anaconda3\envs\tensorflow-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BUFFER_SIZE = 10000
BATCH_SIZE = 256

train_data, test_data = dataset['train'], dataset['test']

train_dataset = train_data.shuffle(BUFFER_SIZE)

train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)

## Build and Compile the model

In [4]:
import tensorflow as tf
import numpy as np

batch_size = 1
timesteps = 20
features = 16
lstm_dim = 8

random_input = np.random.rand(batch_size, timesteps, features)
print(f'Shape of input aray: {random_input}')

lstm = tf.keras.layers.LSTM(lstm_dim)
result = lstm(random_input)
print(f'Shape of output (return_sequences=False): {result.shape}')

lstm_rs = tf.keras.layers.LSTM(lstm_dim, return_sequences=True)
result = lstm_rs(random_input)
print(f'Shape of output (return_sequences=True): {result}')

Shape of input aray: [[[7.24120858e-01 9.89842373e-01 7.42236239e-01 6.33096474e-01
   8.52358067e-01 7.40120140e-01 4.26736285e-01 3.18349578e-01
   2.07346716e-01 2.31191233e-01 4.60718092e-01 5.48794608e-02
   3.73124973e-01 4.73389072e-01 6.85919172e-01 8.55265412e-01]
  [2.19021425e-01 7.20683954e-01 7.90715911e-01 1.38487569e-02
   1.27145569e-01 8.72924769e-01 1.09460623e-01 2.93868853e-01
   1.42857438e-01 6.41116116e-01 3.09926648e-01 5.18128909e-02
   4.00104253e-01 8.31826147e-01 1.29074666e-01 2.12425493e-01]
  [7.19131851e-02 6.86046271e-01 5.71382164e-01 3.35950591e-01
   9.73949333e-01 6.29179540e-01 2.92031177e-01 7.30398301e-01
   6.98918127e-01 6.55904349e-01 8.28780220e-01 8.79651245e-01
   7.29504609e-01 5.28150420e-03 2.09218369e-01 7.36726223e-01]
  [6.74892650e-02 7.58614180e-01 6.35432618e-01 1.30003441e-01
   1.59162428e-02 9.47671743e-02 9.81838581e-01 2.92008079e-01
   8.19572046e-01 1.41042453e-01 7.40702301e-01 8.81712689e-02
   6.83322178e-01 4.06821960e-0

In [5]:
import tensorflow as tf

embedding_dim = 64
lstm1_dim = 64
lstm2_dim = 32
dense_dim = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm1_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm2_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 635329 (2.42 MB)
Trainable params: 635329 

In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train the model

In [7]:
NUM_EPOCHS = 10

history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)


Epoch 1/10


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')
